# Trade Under Pressure



## Import packages

In [ ]:
import functools as ft

import numpy as np
import pandas as pa
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
# Excluded country codes
excluded_country_codes = [
  "NCL", "ZAR", "WLF", "NFK", "PAL", "MNP", "SHN", "COK",
  "BMU", "PYF", "NIU", "ANT", "VGB", "TMP", "GUF", "CYM",
  "TCA", "REU", "FLK", "GLP", "TKL", "MTQ", "AIA", "SPM",
  "SSD", "MNT", "DDR", "BYS", "MNE", "TLS", "LIE", "VAT",
  "CSK", "MSR", "PSE", "DHY", "KOS", "KSV", "RHO", "HVO",
  "VDR", "MCO", "SVU", "XKX", "ASM", "VIR", "SXM", "CUW",
  "MAF", "GUM", "IMN", "MAC", "PCN", "ALI", "YDR", "ATA",
  "CXR", "MID", "SJM", "SPE", "UMI", "SIK", "BAT", "GAZ",
  "BUN", "SGS", "ETF", "PCZ", "TAN", "JTN", "RYU", "PCE",
  "CCK", "BVT", "USP", "KN1", "ATF", "ZPM", "IOT", "MYT",
  "HMD", "SWK", "WAK", "SVR", "ZW1", "PMY", "FRE", "BLX",
  "UNS", "SBH", "NZE", "COG"
]

replace_country_codes = {
  "ROM": {
    "values": ["ROU"]
  },
  "SER": {
    "values": ["SRB"]
  },
  "CHL": {
    "values": ["CHI"]
  },
}

## Data preprocessing
### Dist CEPII

In [ ]:
# Loading and preprocessing Dist CEPII dataset
dist_cepii = pa.read_excel("./data/geographic/dist_cepii.xls",
                           index_col=None, header=0,
                           na_values=[".", "nan", "NaN"], verbose=True, decimal=",",
                           dtype={ "contig": bool, "comlang_off": bool,
                                   "comlang_ethno": bool, "colony": bool,
                                   "smctry": bool }
                           )
dist_cepii = dist_cepii.drop(["comcol", "curcol", "col45"], axis=1)
dist_cepii = dist_cepii.rename(columns={ "iso_o": "origin", "iso_d": "destination" })

Inspect DataFrame properties

In [ ]:
dist_cepii.head(n=50)

In [ ]:
dist_cepii.dtypes

Check how many rows have NaN

In [ ]:
dist_cepii.isna().sum()

Count how many times a country has NaN in Origin and in Destination

In [ ]:
na_rows = dist_cepii[dist_cepii.isna().any(axis=1)][["origin", "destination"]]
# Sanity check
na_rows.isna().sum()

In [ ]:
na_rows[["origin"]].value_counts()

Check how much % of rows are NaN from the total amount

In [ ]:
print("Percentage of NaN rows: ", (na_rows.shape[0] / dist_cepii.shape[0]) * 100, "%")

Drop NaN values

In [ ]:
dist_cepii = dist_cepii.dropna()

In [ ]:
# Sanity check
dist_cepii.isna().any()

Check how many unique countries there are in both origin and destination. Numbers should match.

In [ ]:
print("Unique countries in origin column", dist_cepii["origin"].nunique())
print("Unique countries in destination column", dist_cepii["destination"].nunique())

In [ ]:
dist_cepii.head(5)

Remove unwanted countries

### GSDB V4 Dyadic

Inspect the `GSDB_V4_Dyadic.dta` - this should be a Stata file. What is the difference to `GSDB_V4.csv`?

In [ ]:
gsdb_stata = pa.read_stata("data/sanctions/GSDB_V4_Dyadic.dta")
gsdb_csv = pa.read_csv("data/sanctions/GSDB_V4.csv")

In [ ]:
# Remove sanctions against terrorist organisations
gsdb_stata = gsdb_stata[gsdb_stata["sanctioned_state_iso3"].astype(str) != '']

Retrieve rows that include `case_id` **471**:

In [ ]:
gsdb_stata[gsdb_stata["case_id"].astype(str).str.contains("471")]

In [ ]:
gsdb_stata.head(500)

In [ ]:
gsdb_csv.head(500)

In [ ]:
print("Shape for Stata data: ", gsdb_stata.shape)
print("Shape for XLS data:", gsdb_csv.shape)

In [ ]:
# Column names Stata
gsdb_stata.info()

In [ ]:
gsdb_stata.dtypes

In [ ]:
# Column names XLS
gsdb_csv.info()

In [ ]:
gsdb_csv.dtypes

In [ ]:
gsdb_stata.isna().any()

In [ ]:
(gsdb_stata["sanctioned_state_iso3"].astype(str) == '').sum()

In [ ]:
gsdb_stata[gsdb_stata["sanctioning_state_iso3"] == "YUG"]

In [ ]:
gsdb_stata.head(4)

Try to unify names

### GDP Data

In [ ]:
gdp = pa.read_csv(filepath_or_buffer="data/economic/GDP(currentUSD)_1974-2023.csv", sep=",", na_values=["..", "nan"])

In [ ]:
gdp = gdp.drop(["Series Name", "Series Code", "Country Name"], axis=1)

In [ ]:
gdp = gdp.rename(columns=lambda x: x if not x.endswith("]") else x.split(" ")[0])
gdp = gdp.rename(columns={ "Country Code": "ISO3" })

In [ ]:
gdp.head(10)

In [ ]:
gdp.info()

In [ ]:
gdp.shape

Check how many countries there are in the dataset.

In [ ]:
gdp["ISO3"].nunique()

In [ ]:
gdp["ISO3"].isna().sum()

In [ ]:
gdp = gdp[~gdp["ISO3"].isna()]
print("Shape after removing NaN: ", gdp.shape)

Check countries that are in CEPII, but not in GDP dataset.

In [ ]:
countries_unique_to_cepii = list(set(dist_cepii["origin"].unique()) - set(gdp["ISO3"].unique()))
countries_unique_to_cepii

Check countries that are in GDP, but not in CEPII dataset.

In [ ]:
countries_unique_to_gdp = list(set(gdp["ISO3"].unique()) - set(dist_cepii["origin"].unique()))
countries_unique_to_gdp

### UN Comtrade

In [ ]:
import pandas as pd
import os

# Specify the input and output file paths
input_file = "data/trade/Romania_2020_Plus"  # Update with your input file's path
output_file = "data/trade/Romania_2020_Plus.xlsx"  # Desired output Excel file path

# Determine the file extension to decide how to read the input file
_, ext = os.path.splitext(input_file)
ext = ext.lower()

df = None  # Initialize df


def convert(active):
  if active:
    return
  try:
    if ext in ['.csv', '']:
      # Assume the file is tab-delimited (TSV)
      df = pd.read_csv(input_file, sep='\t', low_memory=False)
    elif ext in ['.xls', '.xlsx']:
      df = pd.read_excel(input_file)
    else:
      raise ValueError(f"Unsupported file extension: {ext}")
    print("File read successfully!")
  except Exception as e:
    print(f"Error reading the input file: {e}")

  # Only attempt conversion if the DataFrame was successfully created
  if df is not None:
    try:
      df.to_excel(output_file, index=False)
      print(f"Conversion successful! The Excel file is saved as: {output_file}")
    except Exception as e:
      print(f"Error writing to Excel: {e}")
  else:
    print("Skipping conversion to Excel because the input file could not be read.")


# Read the input file into a DataFrame
convert(True)


### Rename and remove countries

In [ ]:
gdp = gdp[~gdp.isin(["CHI"]).any(axis=1)]
for key in replace_country_codes.keys():
  for value in replace_country_codes[key]["values"]:
    dist_cepii = dist_cepii.replace(value, key)
    gdp = gdp.replace(value, key)
    gsdb_stata = gsdb_stata.replace(value, key)
    print("Replacing", value, "with", key)

In [ ]:

dist_cepii = dist_cepii[~dist_cepii.isin(excluded_country_codes).any(axis=1)]
gsdb_stata = gsdb_stata[~gsdb_stata.isin(excluded_country_codes).any(axis=1)]
gdp = gdp[~gdp.isin(excluded_country_codes).any(axis=1)]

In [ ]:

# Set distance for SER to the distance of YUG
yug_rows = dist_cepii[dist_cepii.apply(lambda row: row.astype(str).str.contains("YUG").any(), axis=1)].copy()
yug_rows = yug_rows.replace("YUG", "SER")
dist_cepii = pd.concat([dist_cepii, yug_rows], ignore_index=True)



## Collect and analyze country labels in all the data sets

### Country codes

In [ ]:
country_codes = pa.read_excel("./data/geographic/country_codes.xls",
                              dtype={
                                "CountryCode": object,
                              })
country_codes = country_codes[~country_codes.isin(excluded_country_codes).any(axis=1)]

In [ ]:
country_codes = country_codes.rename(columns={ "CountryCode": "UNDS" })

In [ ]:
country_codes.shape

In [ ]:
# Check if all unique
print("All Unique ?: ", country_codes.shape[0] == len(country_codes["ISO3"].unique()))

In [ ]:
country_codes_labels = country_codes["ISO3"]

In [ ]:
np.array(country_codes_labels)

### DIST CEPII Labels

In [ ]:
dist_labels_origin = dist_cepii["origin"].unique()
print(len(dist_labels_origin))
dist_labels_origin

In [ ]:
dist_cepii["origin"].unique()
dist_cepii["origin"].isna().sum()

In [ ]:
dist_labels_dest = dist_cepii["destination"].unique()
print(len(dist_labels_dest))
dist_labels_dest

In [ ]:
dist_cepii["destination"]

In [ ]:
# Check if there are any differences
dist_labels_check = np.array(dist_labels_origin == dist_labels_dest)
print("Any false:", np.any(~dist_labels_check))

In [ ]:
# Sanity check
print("Sum of TRUE values: ", sum(dist_labels_check))
print("Lenght of ALL values: ", len(dist_labels_check))

In [ ]:
dist_labels = dist_labels_origin

In [ ]:
len(dist_labels)

In [ ]:
len(np.unique(dist_labels))

### GSDB Labels

In [ ]:
gsdb_sanctioning_labels = gsdb_stata["sanctioning_state_iso3"].unique()
gsdb_sanctioned_labels = gsdb_stata["sanctioned_state_iso3"].unique()

In [ ]:
print("Number of unique sanctioning: ", len(gsdb_sanctioning_labels))
print("Number of unique sanctioned: ", len(gsdb_sanctioned_labels))

In [ ]:
unique_to_sanctioning = list(set(gsdb_sanctioning_labels) - set(gsdb_sanctioned_labels))
print("Unique to SANCTIONING:", unique_to_sanctioning)

unique_to_sanctioned = list(set(gsdb_sanctioned_labels) - set(gsdb_sanctioning_labels))
print("Unique to SANCTIONED: ", unique_to_sanctioned)

In [ ]:
gsdb_labels = np.union1d(gsdb_sanctioned_labels, gsdb_sanctioning_labels)
print("Total numbers of GSDB labels: ", len(gsdb_labels))

### GDP Labels

In [ ]:
gdp_labels = gdp["ISO3"].unique()
gdp_labels

### Matching

In [ ]:
print("Number of DIST CEPII labels: ", len(dist_labels))
print("Number of GSDB labels: ", len(gsdb_labels))
print("Number of GDP labels: ", len(gdp_labels))
print("Number of country codes: ", len(country_codes_labels))

label_arrays = [dist_labels, gsdb_labels, gdp_labels, country_codes_labels]

intersection_labels = ft.reduce(np.intersect1d, label_arrays)

print("Total number of intersect labels: ", len(intersection_labels))

In [ ]:
intersection_labels

In [ ]:
# Intersection of each set with each other
inter_dist_gsdb = np.intersect1d(gsdb_labels, dist_labels)
print("Intersect DIST CEPII and GSDB labels: ", len(inter_dist_gsdb))
list(set(gsdb_labels) - set(inter_dist_gsdb))

In [ ]:
list(set(dist_labels) - set(intersection_labels))

In [ ]:
list(set(gsdb_labels) - set(intersection_labels))

In [ ]:
list(set(gdp_labels) - set(intersection_labels))

In [ ]:
list(set(country_codes_labels) - set(intersection_labels))

In [ ]:
############
############
############

## Combine Data Sets
Merge the existing data sets into combined data sets for data exploration

## Combine Data Sets
Merge the existing data sets into combined data sets for data exploration

## Combine Data Sets
Merge the existing data sets into combined data sets for data exploration

In [ ]:
# First combine country codes and GDP
country_codes.info()

In [ ]:
country_codes.shape

In [ ]:
gdp.info()

In [ ]:
gdp.shape

In [ ]:
# Inner join GDP and Country Names
gdp_countries = pa.merge(
  gdp[gdp["ISO3"].isin(intersection_labels)],
  country_codes[country_codes["ISO3"].isin(intersection_labels)]
  , on="ISO3", how="inner", validate="m:m")
gdp_countries.info()
list(set(gdp_countries["ISO3"]) - set(intersection_labels))
len(intersection_labels)
len(gdp_countries["ISO3"].unique())

In [ ]:
gdp_countries.shape

In [ ]:
gdp_countries.head(5)

In [ ]:
gdp_countries

## Data Visualization

In [ ]:
sns.set_theme()
sns.color_palette("colorblind")

In [ ]:
gdp_sorted = gdp_countries.sort_values(by=["2023"], ascending=False)
gdp_sorted = gdp_sorted[["Country Name", "2023"]]

In [ ]:
gdp_sorted

In [ ]:
gdp_sorted.describe()

In [ ]:
gdp_year_values = gdp_countries[gdp_countries.columns.difference(["ISO3", "Country Name", "UNDS"])]
gdp_logs = np.log(gdp_year_values)
gdp_scaled = gdp_year_values.apply(
  lambda col: preprocessing.MinMaxScaler().fit_transform(col.values.reshape(-1, 1)).flatten()
)
gdp_scaled

In [ ]:
sns.displot(data=gdp_logs, x="2023")

In [ ]:
sns.boxplot(data=gdp_logs, x="2023")

In [ ]:
gdp_operation = gdp_logs
gdp_max = gdp_operation.max()
gdp_min = gdp_operation.min()
gdp_mean = gdp_operation.mean()
gdp_missing = gdp_operation.isna().sum()
gdp_stats = {
  "Year": gdp_logs.columns,
  "Max": gdp_max.values,
  "Q.75": gdp_operation.quantile(q=0.75).values,
  "Mean": gdp_mean.values,
  "Q.25": gdp_operation.quantile(q=0.25).values,
  "Min": gdp_min.values,
  "Missing": gdp_missing.values,
}
gdp_stats = pa.DataFrame(data=gdp_stats)
gdp_stats = gdp_stats.sort_values("Year").reset_index(drop=True)
gdp_stats["Year"] = gdp_stats["Year"].astype(int)

In [ ]:
long_stats = pd.melt(gdp_stats, id_vars="Year",
                     var_name="Measure", value_name="Value")
long_stats

In [ ]:
def calculate_yearly_change(column_name):
  previous = gdp_stats[column_name].shift(1)
  current = gdp_stats[column_name]

  change_percent = ((current[1:].values - previous[1:].values) / current[1:].values) * 100
  change_percent = pd.Series(change_percent).round(5)
  return change_percent


# Compute change from previous year (1975 has 1974 as "previous year")


yearly_change_percentage = pd.DataFrame({
  "Year": gdp_stats["Year"][1:].values,
  "%-Change-Max": calculate_yearly_change("Max"),
  "%-Change-Min": calculate_yearly_change("Min"),
  "%-Change-Mean": calculate_yearly_change("Mean"),
  "%-Change-Q.25": calculate_yearly_change("Q.25"),
  "%-Change-Q.75": calculate_yearly_change("Q.75"),
})

In [ ]:
long_yearly = pd.melt(yearly_change_percentage, id_vars="Year",
                      var_name="Statistic", value_name="Percent Change")
sns.lineplot(
  data=long_yearly, x="Year", y="Percent Change", hue="Statistic", style="Statistic"
)

In [ ]:
long_yearly

In [ ]:
custom_dashes = {
  'Mean': '',
  'Q.75': (5, 5),
  'Q.25': (5, 5),
  'Max': '',
  'Min': '',
  'Missing': (1, 1),
}
sns.lineplot(
  data=long_stats,
  x="Year",
  y="Value",
  hue="Measure",
  style="Measure",
  dashes=custom_dashes
)

---







# --- ENDE GELÄNDE ---